In [8]:
def nystrom_covariance_estimator(X, indices):
   """
   Estimates the covariance matrix of a data matrix using the Nyström method.

   Parameters:
   -----------
   X : numpy.ndarray
       The input data matrix of shape (n, p), where n is the number of samples
       and p is the number of features.
   indices : numpy.ndarray
       The indices of the landmark points to be used in the Nyström method.

   Returns:
   --------
   Sigma_hat : numpy.ndarray
       The estimated covariance matrix of shape (p, p).
   """
   n, p = X.shape      # n: number of samples, p: number of features

   # Step 1: Use the provided landmark indices
   Y = X[:, indices]      # Y.shape = (n, num_landmarks)

   # Step 2: Compute the orthogonal projection matrix P using the pseudoinverse
   YTY = Y.T @ Y
   YTY_pinv = np.linalg.pinv(YTY)
   P = Y @ (YTY_pinv @ Y.T)    # P.shape = (n, n)

  
   Sigma_hat = X.T @ P @ X / n    # Sigma_hat.shape = (p, p)
  
   return Sigma_hat

In [9]:
import numpy as np

In [19]:
import numpy as np
from scipy import linalg
from scipy.sparse.linalg import eigsh

def recursive_rls_sample(X, num_landmarks=None, replace=False, accelerated_flag=False):
    """
    Perform recursive leverage score sampling on the input matrix X.

    Parameters:
    -----------
    X : numpy.ndarray
        Input matrix of shape (n, p), where n is the number of features and p is the number of data points.
    num_landmarks : int, optional
        Number of landmarks to sample. If None, it's set to ceil(sqrt(p)).
    replace : bool, optional
        Not used in the current implementation.
    accelerated_flag : bool, optional
        If True, use an accelerated version of the algorithm.

    Returns:
    --------
    r_ind : numpy.ndarray
        Indices of the selected columns (landmarks).
    """
    n, p = X.shape  # n is the number of features, p is the number of data points

    # Set the number of landmarks
    if num_landmarks is None:
        s = int(np.ceil(np.sqrt(p)))
    else:
        s = num_landmarks

    # Define the kernel function for columns
    kernel_func = lambda X, col_ind1, col_ind2: (
        X[:, col_ind1].T @ X[:, col_ind2] if len(col_ind2) > 0
        else np.sum(X[:, col_ind1] ** 2, axis=0)
    )

    # Set parameters based on whether acceleration is used
    if not accelerated_flag:
        s_level = s
    else:
        s_level = int(np.ceil(np.sqrt((p * s + s ** 3) / (4 * p))))

    oversamp = np.log(s_level)
    k = int(np.ceil(s_level / (4 * oversamp)))
    n_levels = int(np.ceil(np.log(p / s_level) / np.log(2)))

    # Generate a random permutation of column indices
    perm = np.random.permutation(p)

    # Create a list of decreasing sizes for each level
    l_size = [p]
    for _ in range(n_levels):
        l_size.append(int(np.ceil(l_size[-1] / 2)))

    # Initialize sampling indices and weights
    samp = np.arange(l_size[-1])
    r_ind = perm[samp]
    weights = np.ones(len(r_ind))

    # Compute the diagonal of the kernel matrix
    k_diag = kernel_func(X, np.arange(p), [])

    # Main loop: iterate through levels from bottom to top
    for l in range(n_levels, 0, -1):
        r_ind_curr = perm[:l_size[l - 1]]
        KS = kernel_func(X, r_ind_curr, r_ind)
        SKS = KS[samp, :]
        SKSn = SKS.shape[0]

        # Compute lambda (regularization parameter)
        if k >= SKSn:
            lambda_ = 1e-6
        else:
            diag_SKS = np.diag(SKS)
            diag_sum = np.sum(diag_SKS * weights ** 2)
            SKS_weighted = SKS * weights[np.newaxis, :]
            eigvals = eigsh(
                SKS_weighted @ SKS_weighted.T, k=k, which='LM', return_eigenvectors=False
            )
            eig_sum = np.sum(np.abs(eigvals))
            lambda_ = (diag_sum - eig_sum) / k

        # Compute the inverse of (SKS + lambda * I)
        R = linalg.inv(SKS + np.diag(lambda_ * weights ** (-2)))

        # Compute residuals
        residuals = k_diag[r_ind_curr] - np.sum((KS @ R) * KS, axis=1)

        # Sample columns based on leverage scores
        if l != 1:
            # Compute leverage scores
            levs = np.maximum(
                0,
                np.minimum(
                    1,
                    oversamp * (1 / lambda_) * np.maximum(0, residuals)
                ),
            )
            levs_sum = np.sum(levs)
            
            # Sample columns
            if levs_sum == 0 or np.count_nonzero(levs) < s_level:
                samp = np.random.choice(l_size[l - 1], size=s_level, replace=False)
            else:
                samp = np.random.choice(
                    l_size[l - 1], size=s_level, replace=False, p=levs / levs_sum
                )
            
            # Update weights
            weights = np.sqrt(1 / np.maximum(levs[samp], 1e-12))  # Avoid division by zero
        else:
            # Final sampling step
            levs = np.maximum(
                0, np.minimum(1, (1 / lambda_) * np.maximum(0, residuals))
            )
            levs_sum = np.sum(levs)
            
            # Sample columns
            if levs_sum == 0 or np.count_nonzero(levs) < s:
                samp = np.random.choice(p, size=s, replace=False)
            else:
                samp = np.random.choice(
                    p, size=s, replace=False, p=levs / levs_sum
                )

            # Update weights
            weights = np.sqrt(1 / np.maximum(levs[samp], 1e-12))

        # Update r_ind with the newly sampled columns
        r_ind = r_ind_curr[samp]

    return r_ind

# Example usage:
X = np.random.rand(100, 500)  # 100 features, 500 samples (columns)
selected_columns = recursive_rls_sample(X, num_landmarks=50)
print(selected_columns)


[ 41 242 425 417  76 182   7 295 334 410  78 108  99 372 132 332 173 191
 413 317 430 433  36 137 464 107  69  77 370 391 421 103 161 354 462  75
 485 231 190  44 117   9 341 289 269 125  54  29 228 148]
